# FE 1st

- 확인한 내용들 : hn-eda-prep 커널의 뒷부분 additional topic에서 불친절하게 확인할 수 있음
    - 단지별 가격분포가 얼마나 강력하게 관여되는지
    - 고층 건물이라서 비싼 건지, 집 층수가 높으면 비싼 건지
        - 고층이면 비싼 경향이 확실히 있음
        - 하지만 층수가 더 강력하고 선형적으로 가격에 관여함
        - 더 생각해볼 점 : 층수 정보에서 고층빌딩/단지정보를 제거 해서 층수 정보만 남길 수 있는 방법이 있을까?
    - 거래날짜 정보의 주기성 확인
        - 주기성은 없다고 봄
        - 중요도 : 연 >> 월 > 일
        - 월에서 약간의 주기성이 보이는 듯 하나, 의미있는 변동은 아니라고 봄 <- 날짜 정보를 다룰 때 월을 분리해서 컬럼으로 사용하는 법도 검토해볼만 함
- 바로 할 수 있는 것들만 먼저 적용
    - 가격, 거래 날짜, 법정동코드를 활용한 시구동 핸들링, 면적 정보를 활용한 fe
    - 이미 해본 거 같지만 아직 안 건든 부분도 있어보여서 나름 fe 시도 해봄
- 가격 : log scaling
- 거래 날짜 : 연/월/일, 연월일 단순합성(기존방법)
    - 연월일 스케일링 : 10일 단위를 1로 보고, 월단위를 3, 연단위를 33으로 보고 1~462 범위의 int로 치환
- 면적 정보
    - 추가한 자료 : 공용면적, 전용률
    - 사실 전용면적, 공용면적, 공급면적, 전용률이 단순 사칙연산의 관계이기 때문에, 전용면적과 공용면적만 사용해도 될 듯
- 지역 정보
    - city, 법정코드를 활용해서 서울/부산, 구, 동 정보 조합 시도
- 1/7~8 중에 끝날 내용
    - 아파트 아이디(단지 정보)를 어떻게 Cat 변수로 변환할 것인가 <- 어떻게 변환하느냐가 성능에 큰 영향을 미칠 것 같음
    - Null data를 타겟 값으로 훈련시켜서 null값 채워넣기
- 모델링 쪽에서 확인해줬으면 하는 점
    - 거래 날짜 정보 : 거래 날짜 정보를 여러 가지 방법으로 조합해보았는데,
        중복 컬럼이 되지 않게 (컬럼간 선형 또는 상관이 존재하지 않게) 특성 조합 퍼포먼스 점수랑 feature importance 확인
        - 세 가지 방법 중 가장 나은 것 선택하는 게 좋을 듯
    - 면적정보 : 전용면적과 공용면적만 사용하는게 나은지 확인
    - 지역 정보
        - city, district, town, disTown, cityDisTown의 여러 조합 실험이 필요할 듯
        - 시,구,동 정보가 중복되서 들어가지 않게 조절해야 할 듯
- 오늘 작업 내용의 한계
    - 단일 컬럼과 가격과의 2차원 그래프만 보면서 확인했음

In [2]:
import pandas as pd
import numpy as np
import os, sys, warnings, re, time, gc
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')
sys.path.append(os.path.abspath(os.path.dirname('../')))
gc.collect()

0

In [28]:
%%time
path = './datasets/origin/'
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
subway = pd.read_csv(path+'Subways.csv')
school = pd.read_csv(path+'Schools.csv')
gc.collect()

Wall time: 6.86 s


In [ ]:
# 연산용 코드 모음

train['logPrice']=np.log(train['transaction_real_price'])

train['transYear'] = train['transaction_year_month'].apply(lambda row : int(str(row)[:4]))
train['transMonth'] = train['transaction_year_month'].apply(lambda row : int(str(row)[4:]))
train['transDate'] = train['transaction_date'].apply(lambda row : int(row.split('~')[0]))
test['transYear'] = test['transaction_year_month'].apply(lambda row : int(str(row)[:4]))
test['transMonth'] = test['transaction_year_month'].apply(lambda row : int(str(row)[4:]))
test['transDate'] = test['transaction_date'].apply(lambda row : int(row.split('~')[0]))

def transYMD(row):
    ym = str(row['transaction_year_month'])
    date = str(row['transDate'])
    if len(date)==1 : date = '0'+date
    return int(ym+date)
train['transYMD'] = train.apply(lambda row : transYMD(row), axis=1)
test['transYMD'] = test.apply(lambda row : transYMD(row), axis=1)

def transOdered(row):
    data = str(row)
    year = data[:4]
    month = data[4:6]
    date = data[6:]
    if date == '01' : date = 1
    elif date == '11' : date = 2
    else : date = 3
    month = (int(month)-1)*3
    year = (int(year)-2006)*36
    return year+month+date
train['transOrdered'] = train['transYMD'].apply(lambda row : transOdered(row))
test['transOrdered'] = test['transYMD'].apply(lambda row : transOdered(row))

train['commonArea'] = train.apply(lambda row : row['supply_area']-row['exclusive_use_area'], axis=1)
test['commonArea'] = test.apply(lambda row : row['supply_area']-row['exclusive_use_area'], axis=1)
train['areaRate'] = train.apply(lambda row : row[exclu]/row[supply], axis=1)
test['areaRate'] = test.apply(lambda row : row[exclu]/row[supply], axis=1)

train['district'] = train['address_by_law'].apply(lambda row : int(str(row)[2:5]))
test['district'] = test['address_by_law'].apply(lambda row : int(str(row)[2:5]))
train['town'] = train['address_by_law'].apply(lambda row : int(str(row)[5:8]))
test['town'] = test['address_by_law'].apply(lambda row : int(str(row)[5:8]))
train['disTown'] = train.apply(lambda row : int(str(row['district'])+str(row['town'])), axis=1)
test['disTown'] = test.apply(lambda row : int(str(row['district'])+str(row['town'])), axis=1)
train['cityDisTown'] = train.apply(lambda row : int(str(row['city'])+str(row['disTown'])), axis=1)
test['cityDisTown'] = test.apply(lambda row : int(str(row['city'])+str(row['disTown'])), axis=1)

In [36]:
# path = './datasets'
# train.to_csv(path+'trainFe1st.csv',index=False)
# test.to_csv(path+'testFe1st.csv', index=False)

# 중요 정보

In [4]:
# 가격 로그 스케일링
target = 'logPrice'
train[target]=np.log(train['transaction_real_price'])

In [ ]:
# 아파트 아이디 가격에 따라 정렬해서 int로 캐스팅 : 순서가 있는 카테고리
# apart = 
# uniqId = train['']

# 시간 정보

- 데이터를 살펴보니까, 굳이 날짜정보를 합성해서 쓸 필요가 없을 것 같음
- 중요도 : 연 >> 월 > 일

In [29]:
# 날짜 정보 분리
train['transYear'] = train['transaction_year_month'].apply(lambda row : int(str(row)[:4]))
train['transMonth'] = train['transaction_year_month'].apply(lambda row : int(str(row)[4:]))
train['transDate'] = train['transaction_date'].apply(lambda row : int(row.split('~')[0]))

test['transYear'] = test['transaction_year_month'].apply(lambda row : int(str(row)[:4]))
test['transMonth'] = test['transaction_year_month'].apply(lambda row : int(str(row)[4:]))
test['transDate'] = test['transaction_date'].apply(lambda row : int(row.split('~')[0]))

In [30]:
# 날짜 정보 합성
def transYMD(row):
    ym = str(row['transaction_year_month'])
    date = str(row['transDate'])
    if len(date)==1 : date = '0'+date
    return int(ym+date)
    
train['transYMD'] = train.apply(lambda row : transYMD(row), axis=1)
test['transYMD'] = test.apply(lambda row : transYMD(row), axis=1)
# print(len(train['transYMD'].unique()))
transMin = min(train['transYMD'].min(), test['transYMD'].min())
transMax = min(train['transYMD'].max(),test['transYMD'].max())
print(transMin, transMax)

20060101 20181021


In [31]:
# 날짜정보 정규화
def transOdered(row):
    data = str(row)
    year = data[:4]
    month = data[4:6]
    date = data[6:]
    if date == '01' : date = 1
    elif date == '11' : date = 2
    else : date = 3
    month = (int(month)-1)*3
    year = (int(year)-2006)*36
    return year+month+date
train['transOrdered'] = train['transYMD'].apply(lambda row : transOdered(row))
test['transOrdered'] = test['transYMD'].apply(lambda row : transOdered(row))

# 정규화 / 단순 합성 / 연월일 분리 중 가장 영향력 높은 걸로 하나만 사용하는 게 나을 듯

In [17]:
# train['transOrdered'].max()

462

# 건물/단지 정보

# 집 정보
### 면적 정보

In [32]:
exclu = 'exclusive_use_area'
supply = 'supply_area'
roomid = 'room_id'
# 공용면적
train['commonArea'] = train.apply(lambda row : row[supply]-row[exclu], axis=1)
test['commonArea'] = test.apply(lambda row : row[supply]-row[exclu], axis=1)

In [33]:
# 전용률 : 전용면적 / 전체 면적 = 전용면적의 비율
train['areaRate'] = train.apply(lambda row : row[exclu]/row[supply], axis=1)
test['areaRate'] = test.apply(lambda row : row[exclu]/row[supply], axis=1)

# Null Estimate

In [ ]:
# parking = 'total_parking_capacity_in_site'
# lowest = 'lowest_building_in_sites'
# tallest = 'tallest_building_in_sites'
# front = 'front_door_structure'
room = 'room_count'
bath = 'bathroom_count'


In [ ]:
# 'heat_type', 'heat_fuel'
train['heat_type']

# 위치 정보

In [34]:
# 구 정보 : 3~5
train['district'] = train['address_by_law'].apply(lambda row : int(str(row)[2:5]))
test['district'] = test['address_by_law'].apply(lambda row : int(str(row)[2:5]))

# 동 정보 : 6~8
train['town'] = train['address_by_law'].apply(lambda row : int(str(row)[5:8]))
test['town'] = test['address_by_law'].apply(lambda row : int(str(row)[5:8]))

# 구+동 정보 : 그냥 동정보/구정보 보다 더 정확할 수 있다.
# 왜냐하면, 지역별 땅값은 사실 '구'보다 '동'이 더 영향력이 큼, 구는 좀 뭉뚱그려지는 느낌
train['disTown'] = train.apply(lambda row : int(str(row['district'])+str(row['town'])), axis=1)
test['disTown'] = test.apply(lambda row : int(str(row['district'])+str(row['town'])), axis=1)

# 실험해 볼만한 것 : 앞에 시 정보도 삽입하자
train['cityDisTown'] = train.apply(lambda row : int(str(row['city'])+str(row['disTown'])), axis=1)
test['cityDisTown'] = test.apply(lambda row : int(str(row['city'])+str(row['disTown'])), axis=1)

# 얘들은 Cat Data임을 명심하자

# Drop Cols

In [26]:
# dropCols = ['transaction_date','transaction_year_month', 'room_id','address_by_law','latitude', 'longitude',
#            'supply_area', 'areaRate','total_parking_capacity_in_site', 'lowest_building_in_sites',
#             'tallest_building_in_sites','front_door_structure',]
'''
room id : 데이터 상태가 뭔가 이상함=면적과 불일치함, 집 크기에 관한 정보는 면적 정보로 충분함
시간, 위치 정보 : 세분화가 되어서 중복/불필요해진 데이터
위/경도 : 거의 단지별로 유일함 <- 굳이 중복해서 이용할 필요가 없음
지역 정보 : 가장 좋은 조합만 사용
면적 정보 : 
low importances : 단지 정보에 딸려 있는 정보들 좀 제외하고 쓰면 더 나을 수도
'''
# train = train.drop(columns=dropCols)
# test = test.drop(columns=dropCols)

KeyError: "['areaRate'] not found in axis"

In [35]:
train.columns

Index(['key', 'apartment_id', 'city', 'transaction_year_month',
       'transaction_date', 'year_of_completion', 'exclusive_use_area', 'floor',
       'latitude', 'longitude', 'address_by_law',
       'total_parking_capacity_in_site', 'total_household_count_in_sites',
       'apartment_building_count_in_sites', 'tallest_building_in_sites',
       'lowest_building_in_sites', 'heat_type', 'heat_fuel', 'room_id',
       'supply_area', 'total_household_count_of_area_type', 'room_count',
       'bathroom_count', 'front_door_structure', 'transaction_real_price',
       'transYear', 'transMonth', 'transDate', 'transYMD', 'transOrdered',
       'commonArea', 'areaRate', 'district', 'town', 'disTown', 'cityDisTown'],
      dtype='object')